# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [41]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [42]:
weather_data = pd.read_csv('..\weatherPy\Output\city_data.csv')
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,51.008,98,84,0.45,AU,1619809387
1,Qaanaaq,77.4840,-69.3632,33.134,67,90,2.80,GL,1619809532
2,Rawson,-43.3002,-65.1023,72.752,24,72,3.75,AR,1619809533
3,Rikitea,-23.1203,-134.9692,78.854,77,21,9.28,PF,1619809533
4,Puerto Limón,10.0000,-83.0333,86.000,66,0,4.12,CR,1619809453


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [43]:
locations = weather_data[['Lat','Lng']]
humidity = weather_data['Humidity']

In [44]:
gmaps.configure(api_key=g_key)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,max_intensity=weather_data['Humidity'].max(), dissipating = False,point_radius=3)
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 5 mph.
#cloudiness less than 5.
#Humidity between 30 - 50
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
weather_data_filter_df = weather_data.loc[(weather_data['Humidity']>30)&(weather_data['Humidity']<=50)&(weather_data['Wind Speed']<5)&(weather_data['Cloudiness']==0)&(weather_data['Max Temp']>65)&(weather_data['Max Temp']<=85),:]
weather_data_filter_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
124,Abu Samrah,35.3029,37.1841,71.150,33,0,2.81,SY,1619809593
240,Villarrica,-25.7500,-56.4333,81.410,32,0,2.57,PY,1619809562
318,Necochea,-38.5473,-58.7368,73.994,45,0,4.92,AR,1619809699
329,Concordia,-31.3930,-58.0209,77.000,50,0,3.09,AR,1619809705
479,Itaqui,-29.1253,-56.5531,77.702,37,0,3.55,BR,1619809803
481,Santa Cruz de la Sierra,-17.8000,-63.1667,82.400,39,0,1.54,BO,1619809750
543,Semnan,35.5729,53.3971,71.996,35,0,4.08,IR,1619809845


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
#Create hotel_df dataframe, and store filtered locations to the new df
hotel_df=pd.DataFrame(weather_data_filter_df)
#Add a 'Hotel Name' column
hotel_df['Hotel Name'] =""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
124,Abu Samrah,35.3029,37.1841,71.150,33,0,2.81,SY,1619809593,
240,Villarrica,-25.7500,-56.4333,81.410,32,0,2.57,PY,1619809562,
318,Necochea,-38.5473,-58.7368,73.994,45,0,4.92,AR,1619809699,
329,Concordia,-31.3930,-58.0209,77.000,50,0,3.09,AR,1619809705,
479,Itaqui,-29.1253,-56.5531,77.702,37,0,3.55,BR,1619809803,
481,Santa Cruz de la Sierra,-17.8000,-63.1667,82.400,39,0,1.54,BO,1619809750,
543,Semnan,35.5729,53.3971,71.996,35,0,4.08,IR,1619809845,


In [47]:
#Call google place api and return list of hotels based on the lat/lon pair of the cities in hotel_df
#Take the first result from the api call result and assign the hotel name to the hotel_df
#if no result found, then catch the exception and print out missing result message.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "hotel"
target_type = "hotel"
target_radius = 5000
params = {
        "keyword": target_search,
        "radius":target_radius,
        "type": target_type,
        "key": g_key
    }

#loop through hotel_df and pass api calls
for index, row in hotel_df.iterrows():
    target_coordinates = f"{row['Lat']},{row['Lng']}"
    params['location']=target_coordinates

   
    try:
        #assemble url and make API request
        print(f"Retrieving Results for Index {index}: {target_coordinates}.")
        response = requests.get(base_url, params=params).json()
        # extract results
        results = response['results'][0]['name']

        print(f"Closest hotel near {target_coordinates} is {results}.")
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")    
    print("------------")

Retrieving Results for Index 124: 35.3029,37.1841.
Missing field/result... skipping.
------------
Retrieving Results for Index 240: -25.75,-56.4333.
Closest hotel near -25.75,-56.4333 is Villarrica Palace Hotel.
------------
Retrieving Results for Index 318: -38.5473,-58.7368.
Closest hotel near -38.5473,-58.7368 is Hotel Ñikén Spa & Business Center.
------------
Retrieving Results for Index 329: -31.393,-58.0209.
Closest hotel near -31.393,-58.0209 is Hotel Los Cedros.
------------
Retrieving Results for Index 479: -29.1253,-56.5531.
Closest hotel near -29.1253,-56.5531 is Hotel Contursi.
------------
Retrieving Results for Index 481: -17.8,-63.1667.
Closest hotel near -17.8,-63.1667 is Marriott Santa Cruz de la Sierra Hotel.
------------
Retrieving Results for Index 543: 35.5729,53.3971.
Closest hotel near 35.5729,53.3971 is خانه سمنانی ها.
------------


In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [49]:
# Add marker layer ontop of heat map - issue with tool tip display
markers = gmaps.marker_layer(locations, info_box_content=hotel_info,display_info_box=True)
# Add the layer to the map
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))